In [106]:
import json


file = 'yelp_dataset/yelp_academic_dataset_review.json'
parsed_file = 'yelp_review_small.json'
list_of_reviews_rate = []
with open(file) as f:
    with open(parsed_file, 'w') as outf:
        for i, line in enumerate(f):
            pl = json.loads(line)
            json.dump({"text": pl["text"], "label":pl["stars"]}, outf)
            outf.write('\n')
            if i==5:
                break

In [109]:
import torch   
from torchtext import data

#Reproducing same results
SEED = 2019

#Torch
torch.manual_seed(SEED)

TEXT = data.Field(tokenize='spacy',batch_first=True,include_lengths=True)
LABEL = data.LabelField(dtype = torch.float,batch_first=True)
datafield = {"text": ("text",TEXT),
                     "label": ("label",LABEL)
             }

#loading custom dataset
training_data = data.TabularDataset(path = 'yelp_review_small.json',format = 'json',fields = datafield)

#print preprocessed text
print(len(training_data.examples))


5


In [110]:
import random
train_data, test_data = training_data.split(split_ratio=0.7, random_state = random.seed(SEED))

In [112]:
#initialize glove embeddings
TEXT.build_vocab(train_data,min_freq=3,vectors = "glove.6B.100d")  
LABEL.build_vocab(train_data)

#No. of unique tokens in text
print("Size of TEXT vocabulary:",len(TEXT.vocab))

#No. of unique tokens in label
print("Size of LABEL vocabulary:",len(LABEL.vocab))

#Commonly used words
print(TEXT.vocab.freqs.most_common(10))  

#Word dictionary
print(TEXT.vocab.stoi)  

.vector_cache/glove.6B.zip: 862MB [06:55, 2.07MB/s]                               
100%|█████████▉| 399999/400000 [00:17<00:00, 23346.10it/s]


Size of TEXT vocabulary: 74
Size of LABEL vocabulary: 3
[('.', 40), ('the', 37), (',', 29), ('I', 20), ('and', 18), ('to', 18), ('a', 17), ('"', 14), ('is', 11), ('of', 11)]
defaultdict(<bound method Vocab._default_unk_index of <torchtext.vocab.Vocab object at 0x130b3f7d0>>, {'<unk>': 0, '<pad>': 1, '.': 2, 'the': 3, ',': 4, 'I': 5, 'and': 6, 'to': 7, 'a': 8, '"': 9, 'for': 10, 'is': 11, 'of': 12, 'it': 13, '\n\n': 14, 'this': 15, 'was': 16, '\n': 17, "'s": 18, 'he': 19, 'in': 20, '!': 21, '?': 22, 'my': 23, '*': 24, 'just': 25, 'with': 26, 'you': 27, 'cut': 28, 'not': 29, ';': 30, 'That': 31, 'The': 32, 'are': 33, 'around': 34, 'art': 35, 'as': 36, 'at': 37, 'but': 38, 'do': 39, 'go': 40, 'had': 41, 'have': 42, 'me': 43, 'on': 44, 'or': 45, 'that': 46, 'there': 47, 'who': 48, '(': 49, ')': 50, 'It': 51, 'When': 52, 'all': 53, 'any': 54, 'down': 55, 'from': 56, 'gallery': 57, 'hair': 58, 'haircut': 59, 'like': 60, "n't": 61, 'no': 62, 'out': 63, 'place': 64, 'real': 65, 'said': 66, 'sh

In [113]:
#set batch size
BATCH_SIZE = 64

#Load an iterator
train_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_key = lambda x: len(x.text),
    sort_within_batch=True)

/Users/ilannaiman/opt/anaconda3/envs/torchDP/lib/python3.7/site-packages/torchtext/data/iterator.py:48: UserWarning: BucketIterator class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.
  warnings.warn('{} class will be retired in the 0.8.0 release and moved to torchtext.legacy. Please see 0.7.0 release notes for further information.'.format(self.__class__.__name__), UserWarning)
